In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import datetime as dt
import seaborn as sns


%pylab inline
%matplotlib inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
order = pd.read_csv('/Users/siang/Documents/107-1/資料科學實務/data_ver01/order.csv', sep =',', encoding='utf8',
                    dtype={'order_id':object,'group_id':object})

In [3]:
group = pd.read_csv('/Users/siang/Documents/107-1/資料科學實務/data_ver01/group.csv',sep = ',',encoding='utf8',dtype={'order_id':object,'group_id':object})

In [4]:
order.order_id.count()

396915

In [8]:
training = pd.read_csv('/Users/siang/Documents/107-1/資料科學實務/data_ver01/training-set.csv',
                       sep = ',',encoding='utf8',dtype={'order_id':object})

In [9]:
training.order_id.count()

297020

In [60]:
order[order['order_id']==43556]

,order_id,deal_or_not,group_id,order_date,source_1,source_2,unit,people_amount


In [56]:
print(group[group['group_id']==54856].begin_date)
print(order[order['order_id']==43556].order_date)

Series([], Name: begin_date, dtype: object)
Series([], Name: order_date, dtype: object)


In [10]:
print(order.dtypes)
print(group.dtypes)
print(training.dtypes)

order_id         object
group_id         object
order_date       object
source_1          int64
source_2          int64
unit              int64
people_amount     int64
dtype: object
group_id          object
sub_line           int64
area               int64
days               int64
begin_date        object
price              int64
product_name      object
promotion_prog    object
dtype: object
order_id       object
deal_or_not     int64
dtype: object


In [11]:
len(set(training.order_id).intersection(order.order_id))

297020

In [12]:
order = pd.merge(training, order, on = 'order_id', how='inner')

In [14]:
order.isna().sum()

order_id         0
deal_or_not      0
group_id         0
order_date       0
source_1         0
source_2         0
unit             0
people_amount    0
dtype: int64

In [15]:
#將group的begin date merge 到 order
temp = pd.merge(order, group[['group_id','begin_date']], on='group_id', how='left')

In [16]:
temp['order_date'] = temp['order_date'].astype('datetime64')
temp['begin_date'] = temp['begin_date'].astype('datetime64')

In [17]:
temp.head()

,order_id,deal_or_not,group_id,order_date,source_1,source_2,unit,people_amount,begin_date
0,136100,0,63695,2017-01-05,1,1,1,2,2017-03-22
1,140370,0,63695,2016-11-09,1,1,2,1,2017-03-22
2,211009,0,63695,2017-01-20,1,1,3,2,2017-03-22
3,41571,1,53481,2017-03-14,1,1,4,4,2017-05-17
4,64125,1,53481,2017-03-27,2,1,5,1,2017-05-17


In [18]:
temp['wait_duration'] = (temp['begin_date'] - temp['order_date']).dt.days

In [19]:
temp.head()

,order_id,deal_or_not,group_id,order_date,source_1,source_2,unit,people_amount,begin_date,wait_duration
0,136100,0,63695,2017-01-05,1,1,1,2,2017-03-22,76
1,140370,0,63695,2016-11-09,1,1,2,1,2017-03-22,133
2,211009,0,63695,2017-01-20,1,1,3,2,2017-03-22,61
3,41571,1,53481,2017-03-14,1,1,4,4,2017-05-17,64
4,64125,1,53481,2017-03-27,2,1,5,1,2017-05-17,51


In [20]:
temp['order_date_weekday'] = temp['order_date'].dt.dayofweek + 1

In [21]:
temp['order_date_is_weekend'] = np.where((temp['order_date_weekday']==6)|(temp['order_date_weekday']==7), 1 ,0)

In [22]:
temp['begin_date_weekday'] = temp['begin_date'].dt.dayofweek + 1

In [23]:
temp['begin_date_is_weekend'] = np.where((temp['begin_date_weekday']==6)|(temp['begin_date_weekday']==7), 1 ,0)

In [24]:
temp.to_csv('order_with_group_begin_date.csv', sep = ",", encoding = 'utf8', index = False)

In [26]:
temp.isna().sum()

order_id                 0
deal_or_not              0
group_id                 0
order_date               0
source_1                 0
source_2                 0
unit                     0
people_amount            0
begin_date               0
wait_duration            0
order_date_weekday       0
order_date_is_weekend    0
begin_date_weekday       0
begin_date_is_weekend    0
dtype: int64

In [55]:
temp.order_id.count()

297020

In [44]:
unit_group_count = temp.groupby(['unit','group_id']).agg('count')
unit_group_count.reset_index(inplace = True)
unit_group_count = unit_group_count[['unit','group_id','order_id']]
unit_group_count = unit_group_count.rename(columns ={'order_id':'unit_group_count'})

In [46]:
unit_group_count.unit_group_count.describe()

count    183252.000000
mean          1.620828
std           1.357105
min           1.000000
25%           1.000000
50%           1.000000
75%           2.000000
max          51.000000
Name: unit_group_count, dtype: float64

In [48]:
unit_group_count = pd.merge(temp, unit_group_count, on=['unit','group_id'], how='left')

In [50]:
unit_group_count = unit_group_count[['order_id','unit_group_count']]

In [52]:
unit_group_count.unit_group_count.isna().sum()

0

In [54]:
unit_group_count.to_csv('unit_group_count.csv', sep=',', index = False, encoding='utf8')